# Selection sets

In [ ]:
#r "C:\Autodesk\ObjectARX_for_AutoCAD_2021_Win_64bit_dlm\inc\AcCoreMgd.dll"
#r "C:\Autodesk\ObjectARX_for_AutoCAD_2021_Win_64bit_dlm\inc\AcMgd.dll"
using ACAD = Autodesk.AutoCAD.ApplicationServices.Application;
using Autodesk.AutoCAD.DatabaseServices;
using Autodesk.AutoCAD.EditorInput;
using Autodesk.AutoCAD.Geometry;
using Autodesk.AutoCAD.Runtime;

## PICKFIRST
        • PICKFIRST system variable must be set to 1
        • UsePickSet command flag must be defined with the command that should use the Pickfirst selection set
        • Call the SelectImplied method to obtain the PickFirst selection set

#### GetSelection
Prompts the user to pick objects from the screen.

#### SelectAll
Selects all objects in the current space in which are not locked or frozen.

#### SelectCrossingPolygon
Selects objects within and crossing a polygon defined by specifying points. The polygon can be any shape but cannot cross or touch itself.

#### SelectCrossingWindow
Selects objects within and crossing an area defined by two points.

#### SelectFence
Selects all objects crossing a selection fence. Fence selection is similar to crossing polygon selection except that the fence is not closed, and a fence can cross itself.

#### SelectLast
Selects the last object created in the current space.

#### SelectPrevious
Selects all objects selected during the previous Select objects: prompt.

#### SelectWindow
Selects all objects completely inside a rectangle defined by two points.

#### SelectWindowPolygon
Selects objects completely inside a polygon defined by points. The polygon can be any shape but cannot cross or touch itself.

#### SelectAtPoint
Selects objects passing through a given point and places them into the active selection set.

#### SelectByPolygon
Selects objects within a fence and adds them to the active selection set.

In [ ]:

[CommandMethod("CheckSelectionSets", CommandFlags.UsePickSet)] public static void SelectSet()
{
    Editor ed = ACAD.DocumentManager.MdiActiveDocument.Editor;
    PromptSelectionResult acSSPrompt = ed.SelectImplied();
    // Clear the PickFirst selection set
    ObjectId[] idarrayEmpty = new ObjectId[0];
    ed.SetImpliedSelection(idarrayEmpty);
    // Request for objects to be selected in the drawing area
    acSSPrompt = ed.GetSelection();
    
    //selects the objects within and that intersect a crossing window.
    acSSPrompt = ed.SelectCrossingWindow(new Point3d(2, 2, 0), new Point3d(10, 8, 0));
}

## Add To or Merge Multiple Selection Sets
    Merge multiple selection sets be creating an ObjectIdCollection object and then adding the object ids from multiple selection sets together.

In [ ]:
//Add To or Merge Multiple Selection Sets
SelectionSet acSSet1;
acSSet1 = acSSPrompt.Value;
ObjectIdCollection coll = new ObjectIdCollection(acSSet1.GetObjectIds());
SelectionSet acSSet2 = acSSPrompt.Value;
if (coll.Count == 0)
{
    coll = new ObjectIdCollection(acSSet2.GetObjectIds());  
}
else
{
    foreach (ObjectId id in acSSet2.GetObjectIds())
    {
        coll.Add(id);
    }
}

## Define Rules for Selection Filters
    - A selection filter list can be used to filter selected objects by properties or type.
    - Selection filters are composed of pairs of arguments in the form of TypedValues
        - First = type of filter (Object)
        - Second = value you are filtering on (Circle)

![names](SelFilters.PNG)

### Specify a single selection criterion for a selection set

In [ ]:
// Create a TypedValue array to define the filter criteria
TypedValue[] acTypValAr = new TypedValue[1];
acTypValAr.SetValue(new TypedValue((int)DxfCode.Start, "CIRCLE"), 0);
// Assign the filter criteria to a SelectionFilter object
SelectionFilter acSelFtr = new SelectionFilter(acTypValAr);
PromptSelectionResult acSSPrompt = ed.GetSelection(acSelFtr);


### Specify Multiple Criteria in a Selection Filter
    
#### Two criterion to filter selected objects by: the object must be a circle and it must reside on layer 0.

In [ ]:
// Create a TypedValue array to define the filter criteria
TypedValue[] acTypValAr = new TypedValue[3];
acTypValAr.SetValue(new TypedValue((int)DxfCode.Color, 5), 0);
acTypValAr.SetValue(new TypedValue((int)DxfCode.Start, "CIRCLE"), 1);
acTypValAr.SetValue(new TypedValue((int)DxfCode.LayerName, "0"), 2);
// Assign the filter criteria to a SelectionFilter object
SelectionFilter acSelFtr = new SelectionFilter(acTypValAr);

### Add Complexity to Your Filter List Conditions
     Relational operations (for example, the radius of a circle must be greater than or equal to 5.0)
     Use a -4 DXF code or the constant DxfCode.Operator to indicate a relational operator in a selection filter. 
          The operator is expressed as a string.

| Operator | Description |
| --- | --- |
| "&" | Bitwise AND (integer groups only) |
| "&=" | Bitwise masked equals (integer groups only) |

![names](FilterLogicalOprs.PNG)

#### Select a circle whose radius is greater than or equal to 5.0

In [ ]:
// Create a TypedValue array to define the filter criteria
TypedValue[] acTypValAr = new TypedValue[3];
acTypValAr.SetValue(new TypedValue((int)DxfCode.Start, "CIRCLE"), 0);
acTypValAr.SetValue(new TypedValue((int)DxfCode.Operator, ">="), 1);
acTypValAr.SetValue(new TypedValue(40, 5), 2);
// Assign the filter criteria to a SelectionFilter object
SelectionFilter acSelFtr = new SelectionFilter(acTypValAr);

#### Select either Text or MText

In [ ]:
/ Create a TypedValue array to define the filter criteria
TypedValue[] acTypValAr = new TypedValue[4];
acTypValAr.SetValue(new TypedValue((int)DxfCode.Operator, "<or"), 0);
acTypValAr.SetValue(new TypedValue((int)DxfCode.Start, "TEXT"), 1);
acTypValAr.SetValue(new TypedValue((int)DxfCode.Start, "MTEXT"), 2);
acTypValAr.SetValue(new TypedValue((int)DxfCode.Operator, "or>"), 3);
// Assign the filter criteria to a SelectionFilter object
SelectionFilter acSelFtr = new SelectionFilter(acTypValAr);

### Use Wild-Card Patterns in Selection Set Filter Criteria